In [64]:
import pandas as pd
import numpy as np
import math

df = pd.read_csv('CTR.txt',delimiter=':',header = None, names=['num article','rep en 5d','taux de clics'])
fich = df.get_values()
fich = generate_data(fich)

In [67]:
#random a chaque ligne
def generate_data(fichier):
    article = {}
    for ligne in fichier:
        #numero de l'article
        num = ligne[0]
        #annonceurs
        annonceurs = ligne[1]
        annonceurs = annonceurs.split(';')
        annonceurs = np.array(annonceurs,dtype=float)
        #contexte
        contexte = ligne[2]
        contexte = contexte.split(';')
        contexte = np.array(contexte,dtype=float)
        article[num] = [annonceurs,contexte]
    return article
        
#renvoi le numero de l'action
def randomBaseline(ligne):    
    return np.random.randint(0,len(ligne[1]))

#opti sur chaque ligne
def optimaleBaseline(ligne):
#     print(ligne[1])
    #return index de l'arg max => pub numero ...
    return np.argmax(ligne[1])

# meilleur selon la moyenne cumulé 
# tout le temps le meme 
def staticBestBaseline(fich):
    sum_cumule = np.zeros(10, dtype=float)
    for i in fich:
        sum_cumule += fich[i][1]
    return np.argmax(sum_cumule)
        
    
optimaleBaseline(fich[0])

staticBestBaseline(fich[0])


randomBaseline(fich[0])

6

In [84]:

# on s'arrete quand on veut c'est de l'apprentissage 
# μbi,x =1/x Somme(s=1,x) {gi,s} avec gi,s ls-ieme reward obtenu par le bras i
def borne(means,count,t):
    res = means + np.sqrt(2*np.log(t)/count)
#     print(res)
    return res

# means[action] = (1 - 1/count[action]) * means[action] + (1/count[action]) * reward

means = np.zeros(5)
count_action = np.zeros(5)

r = 0
action_staticBest = staticBestBaseline(fich)
reward_staticBest = 0
reward_random = 0
reward_optimale = 0

for t in range(10**2):
    if t < 5 :
        action = t
    else :
        action = np.argmax(borne(means,t,count_action))
        
    index = t%len(fich)
    
    action_random = randomBaseline(fich[index])
    reward_random += fich[index][1][action_random]
    
    
    reward_staticBest += fich[index][1][action_staticBest]
    
    action_optimale = optimaleBaseline(fich[index])
#     print(action_optimale)
    reward_optimale += fich[index][1][action_optimale]
    
    reward_action = fich[index][1][action]

    r += reward_action
    count_action[action] += 1
    means[action] = (1 - 1/count_action[action]) * means[action] + (1/count_action[action]) * reward_action

    
print(reward_random)
print(reward_optimale)
print(reward_staticBest)
print(r)
    

6.41142996272
30.1437602403
25.8029484235
25.582651724
